This is the final project with all of the experiments stripped out.

### load modules

In [1]:
import sys
import pickle
sys.path.append("../tools/")

import pandas as pd
import numpy as np

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import Imputer
                                                
from pprint import pprint
from sklearn.tree import DecisionTreeClassifier

### import data into Pandas DataFrame

In [2]:
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)
df = pd.DataFrame(data_dict)
df = df.transpose()
df.head()

,bonus,deferral_payments,deferred_income,director_fees,email_address,exercised_stock_options,expenses,from_messages,from_poi_to_this_person,from_this_person_to_poi,...,long_term_incentive,other,poi,restricted_stock,restricted_stock_deferred,salary,shared_receipt_with_poi,to_messages,total_payments,total_stock_value
ALLEN PHILLIP K,4175000,2869717,-3081055,NaN,phillip.allen@enron.com,1729541,13868,2195,47,65,...,304805,152,False,126027,-126027,201955,1407,2902,4484442,1729541
BADUM JAMES P,NaN,178980,NaN,NaN,NaN,257817,3486,NaN,NaN,NaN,...,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,182466,257817
BANNANTINE JAMES M,NaN,NaN,-5104,NaN,james.bannantine@enron.com,4046157,56301,29,39,0,...,NaN,864523,False,1757552,-560222,477,465,566,916197,5243487
BAXTER JOHN C,1200000,1295738,-1386055,NaN,NaN,6680544,11200,NaN,NaN,NaN,...,1586055,2660303,False,3942714,NaN,267102,NaN,NaN,5634343,10623258
BAY FRANKLIN R,400000,260455,-201641,NaN,frank.bay@enron.com,NaN,129142,NaN,NaN,NaN,...,NaN,69,False,145796,-82782,239671,NaN,NaN,827696,63014


### Task 1: Select Features

In [3]:
df = df.drop('email_address', axis=1)
df = df.astype(float)

### Task 2: Eliminate Outliers/Missing Data

In [4]:
df = df.drop('TOTAL')
df = df.drop("THE TRAVEL AGENCY IN THE PARK")
df = df.drop("loan_advances", axis=1)
#letting the imputer deal with missing data
#df.fillna(0, inplace=True)

### Task 3: Create New Features

In [5]:
df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)

#### sklearn prep: features_list

In [6]:
features_list = list(df.columns)
features_list.remove('poi')

In [7]:
len(features_list)

19

### Task 4: Try a variety of classifiers

First I am going to try the GridSearchCV tutorial with the data in the machine learning class. 

From page 1, I start with the data in a pandas data frame. 

We don't have any categorical variables besides the dependent variable so I will not have to use the sklearn.preprocessing.LabelEncoder() or the pandas.get_dummies() functions. 

The data is in a pandas data frame and has not been split into training or testing data sets. So what we have to do is create the features and labels. 

In [8]:
labels = df['poi']

In [9]:
features = df[features_list]

In [10]:
from sklearn import cross_validation
features_train, features_test, labels_train, labels_test = \
    cross_validation.train_test_split(features, labels, test_size=0.3, random_state=42)

### Grid Search Cross Validation
So we make lists of the parameters of the model that we want to search through. Then there are parameters of the GridSearchCV model itself. Then we fit the model, test its predictions and print out the best estimator and its parameters. 

Ok, I deleted the grid searches because they were working and I thought I was ready to combine them with pipelines, but now I am messing up how to combine them with pipelines. 

### Pipelines
The next step is to use all of this grid searching and pipelines working on the Gaussian Naive Bayes model. That will get me the best parameters.

First, I look up what the possible parameters are for the GaussianNB. Of course, this will also require that I normalize the data and do the Principle Components analysis on the data before it goes to the model so I will also have to do some data preparation and build a pipeline. 

The Gaussian Naive Bayes model takes the parameters of--ok, well it doesn't really have any parameters to take. It just wants the normalized data and the principle components if you are doing that. So, we will skip the Gaussian for now and, following the tutorial, add in the pipeline for the decision tree classifier. 

Ok, so I have gotten rid of all the experiments and tests done using the data with the missing values cleaned by the Pandas functions and am now going to do all the data cleaning in the pipeline so I can compare different imputation strategies. 

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# imp = Imputer(missing_values='NaN', strategy='0')
# features_train = imp.fit_transform(features_train)
# features_test = imp.fit_transform(features_test)

Pipeline = Pipeline([
        ('imp', Imputer(missing_values='NaN', strategy='median')),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', DecisionTreeClassifier(random_state = 53))
    ])

pipeline = Pipeline.fit(features_train, labels_train)

print "Pipeline Accuracy Score:"
print pipeline.score(features_test, labels_test)

print "\nPipeline parameters:"
pprint(pipeline.get_params())

clf_pipeline_pred = pipeline.predict(features_test)
print "\n\nRecall Score:", recall_score(labels_test, clf_pipeline_pred)

Pipeline Accuracy Score:
0.795454545455

Pipeline parameters:
{'clf': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=53, splitter='best'),
 'clf__class_weight': None,
 'clf__criterion': 'gini',
 'clf__max_depth': None,
 'clf__max_features': None,
 'clf__max_leaf_nodes': None,
 'clf__min_samples_leaf': 1,
 'clf__min_samples_split': 2,
 'clf__min_weight_fraction_leaf': 0.0,
 'clf__presort': False,
 'clf__random_state': 53,
 'clf__splitter': 'best',
 'imp': Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0),
 'imp__axis': 0,
 'imp__copy': True,
 'imp__missing_values': 'NaN',
 'imp__strategy': 'median',
 'imp__verbose': 0,
 'pca': PCA(copy=True, n_components=None, whiten=False),
 'pca__copy': True,
 'pca__n_components': None,
 'pca__whiten': False,
 'std': Stand

Now we are going to run the grid search with the pipeline put in as the estimator. The things we want to change in the pipeline will be put in through the grid search. 

One thing that is a little confusing is that we specify one imputation strategy in in the pipeline and another in the param_grid. I can't do both, can I? Is that part of my problem? 

In [23]:
#Massive Accuracy Drop, Don't understand missing values
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

Pipeline = Pipeline([
        ('imp', Imputer()),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', DecisionTreeClassifier(random_state = 53))
    ])


param_grid = {'imp__strategy':['median', 'most_frequent'],
              'clf__min_samples_split': [1, 2, 3, 4, 5, 6, 10, 15, 20, 25, 30],
              'clf__max_depth': [4, 5, 6, 7, 8],
              'pca__n_components': [2,3,4,5,6,7]}

#create the GridSearchCV object using param_grid
gridCV_object = GridSearchCV(estimator = Pipeline, 
                                        param_grid = param_grid,
                                        cv = 3)

gridCV_object.fit(features_train, labels_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('std', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, n_components=None, whiten=False)), ('clf', DecisionTreeClassifier(class_weight=None, criterion='g...lit=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=53, splitter='best'))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'pca__n_components': [2, 3, 4, 5, 6, 7], 'clf__max_depth': [4, 5, 6, 7, 8], 'clf__min_samples_split': [1, 2, 3, 4, 5, 6, 10, 15, 20, 25, 30], 'imp__strategy': ['median', 'most_frequent']},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [24]:
print "Best parameters from the grid search:", pprint(gridCV_object.best_params_)

Best parameters from the grid search:{'clf__max_depth': 4,
 'clf__min_samples_split': 15,
 'imp__strategy': 'most_frequent',
 'pca__n_components': 4}
 None


In [25]:
clf_gridCV = gridCV_object.best_estimator_

In [26]:
print "\nBest estimator accuracy:", clf_gridCV.score(features_test, labels_test)


Best estimator accuracy: 0.840909090909


In [27]:
clf_gridCV_pred = clf_gridCV.predict(features_test)
print "\n\nRecall Score:", recall_score(labels_test, clf_gridCV_pred)



Recall Score: 0.2


Now the recall score is 0? What did I do wrong? The accuracy is higher but the recall has dropped to 0. That doesn't seem possible. 

UPDATE: We had the coach chat and then the computer shut down so I lost a lot of what we said, but I think I got a lot of the idea. Now I ran the model again with slight changes. I took cv from 8 down to 3. Seemed to have a big effect. Also, he suggested that we switch off the numbers and use the stratified shuffle split instead. 

Now I am going to do it again and add the stratified shuffle split and make the search criteria recall. 

In [28]:
#Now using stratified shuffle split
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.cross_validation import StratifiedShuffleSplit
cross_validator = StratifiedShuffleSplit(y = labels_train, random_state = 0)

Pipeline = Pipeline([
        ('imp', Imputer()),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', DecisionTreeClassifier(random_state = 53))
    ])


param_grid = {'imp__strategy':['median', 'most_frequent'],
              'clf__min_samples_split': [1, 2, 3, 4, 5, 6, 10, 15, 20, 25, 30],
              'clf__max_depth': [4, 5, 6, 7, 8],
              'pca__n_components': [2,3,4,5,6,7]}

#create the GridSearchCV object using param_grid
gridCV_object = GridSearchCV(estimator = Pipeline, 
                                        param_grid = param_grid,
                                        cv = cross_validator)

#fit the model
gridCV_object.fit(features_train, labels_train)

#print the best model's parameters
print "Best parameters from the grid search:", pprint(gridCV_object.best_params_)

#instantiate the best model
clf_gridCV = gridCV_object.best_estimator_

#print out the best model's accuracy
print "\nBest estimator accuracy:", clf_gridCV.score(features_test, labels_test)

#get predictions and print out recall score
clf_gridCV_pred = clf_gridCV.predict(features_test)
print "\n\nRecall Score:", recall_score(labels_test, clf_gridCV_pred)

Best parameters from the grid search:{'clf__max_depth': 4,
 'clf__min_samples_split': 4,
 'imp__strategy': 'most_frequent',
 'pca__n_components': 4}
 None

Best estimator accuracy: 0.840909090909


Recall Score: 0.2


Now we add the recall argument for the scoring function.

In [29]:
#Now using stratified shuffle split
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.cross_validation import StratifiedShuffleSplit
cross_validator = StratifiedShuffleSplit(y = labels_train, random_state = 0)

Pipeline = Pipeline([
        ('imp', Imputer()),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', DecisionTreeClassifier(random_state = 53))
    ])


param_grid = {'imp__strategy':['median', 'most_frequent'],
              'clf__min_samples_split': [1, 2, 3, 4, 5, 6, 10, 15, 20, 25, 30],
              'clf__max_depth': [4, 5, 6, 7, 8],
              'pca__n_components': [2,3,4,5,6,7]}

#create the GridSearchCV object using param_grid
gridCV_object = GridSearchCV(estimator = Pipeline, 
                                        param_grid = param_grid,
                                        scoring = 'recall',
                                        cv = cross_validator)

#fit the model
gridCV_object.fit(features_train, labels_train)

#print the best model's parameters
print "Best parameters from the grid search:", pprint(gridCV_object.best_params_)

#instantiate the best model
clf_gridCV = gridCV_object.best_estimator_

#print out the best model's accuracy
print "\nBest estimator accuracy:", clf_gridCV.score(features_test, labels_test)

#get predictions and print out recall score
clf_gridCV_pred = clf_gridCV.predict(features_test)
print "\n\nRecall Score:", recall_score(labels_test, clf_gridCV_pred)

Best parameters from the grid search:{'clf__max_depth': 4,
 'clf__min_samples_split': 1,
 'imp__strategy': 'median',
 'pca__n_components': 2}
 None

Best estimator accuracy: 0.840909090909


Recall Score: 0.2


Now we can switch out the tree classifier for the random forest classifier. And we add in the 'clf__n_estimators': [2,5,10,15] which is a key parameter for the random forest model. I have also removed the min_samples_split parameter from the param_grid to speed things up. 

In [32]:
#Now using stratified shuffle split
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

from sklearn.cross_validation import StratifiedShuffleSplit
cross_validator = StratifiedShuffleSplit(y = labels_train, random_state = 0)

Pipeline = Pipeline([
        ('imp', Imputer()),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', RandomForestClassifier(random_state = 53))
    ])


param_grid = {'imp__strategy':['median', 'most_frequent'],
              #'clf__min_samples_split': [2, 3, 4, 5, 6, 10, 15, 20],
              'clf__max_depth': [4, 5, 6, 7, 8],
              'clf__n_estimators': [2,5,10,15],
              'pca__n_components': [2,3,4,5,6,7]}

#create the GridSearchCV object using param_grid
gridCV_object = GridSearchCV(estimator = Pipeline, 
                                        param_grid = param_grid,
                                        scoring = 'recall',
                                        cv = cross_validator)

#fit the model
gridCV_object.fit(features_train, labels_train)

#print the best model's parameters
print "Best parameters from the grid search:", pprint(gridCV_object.best_params_)

#instantiate the best model
clf_gridCV = gridCV_object.best_estimator_

#print out the best model's accuracy
print "\nBest estimator accuracy:", clf_gridCV.score(features_test, labels_test)

#get predictions and print out recall score
clf_gridCV_pred = clf_gridCV.predict(features_test)
print "\n\nRecall Score:", recall_score(labels_test, clf_gridCV_pred)

Best parameters from the grid search:{'clf__max_depth': 4,
 'clf__n_estimators': 2,
 'imp__strategy': 'most_frequent',
 'pca__n_components': 6}
 None

Best estimator accuracy: 0.863636363636


Recall Score: 0.0


Now the accuracy has gone up but the recall has gone to zero. That seems almost unbelievable. After all, I have made recall the scoring function. 

Max depth has was very low at 4 so maybe we should truncate the parameter list there. And we should also reduce the number of estimators we look through. 

For some reason the imputation strategy has flipped over to 'most_frequent' rather than 'median'. I have no idea why. Anyway, the next step is to do what? I have lost the recall that I had. Where do I go from here? 

I am going to give up on the random forest classifiers for now and go to the Gaussian model that did well for me before. 

In [34]:
#Now using stratified shuffle split
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
#from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.cross_validation import StratifiedShuffleSplit
cross_validator = StratifiedShuffleSplit(y = labels_train, random_state = 0)

Pipeline = Pipeline([
        ('imp', Imputer()),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', GaussianNB())
    ])


param_grid = {'imp__strategy':['median', 'most_frequent'],
              #'clf__min_samples_split': [2, 3, 4, 5, 6, 10, 15, 20],
              #'clf__max_depth': [4, 5, 6, 7, 8],
              #'clf__n_estimators': [2,5,10,15],
              'pca__n_components': [2,3,4,5,6,7]}

#create the GridSearchCV object using param_grid
gridCV_object = GridSearchCV(estimator = Pipeline, 
                                        param_grid = param_grid,
                                        scoring = 'recall',
                                        cv = cross_validator)

#fit the model
gridCV_object.fit(features_train, labels_train)

#print the best model's parameters
print "Best parameters from the grid search:", pprint(gridCV_object.best_params_)

#instantiate the best model
clf_gridCV = gridCV_object.best_estimator_

#print out the best model's accuracy
print "\nBest estimator accuracy:", clf_gridCV.score(features_test, labels_test)

#get predictions and print out recall score
clf_gridCV_pred = clf_gridCV.predict(features_test)
print "\n\nRecall Score:", recall_score(labels_test, clf_gridCV_pred)

Best parameters from the grid search:{'imp__strategy': 'median', 'pca__n_components': 10}
 None

Best estimator accuracy: 0.818181818182


Recall Score: 0.2


So I am going to try to increase the number of dimensions we look through to 15.

In [35]:
#Now using stratified shuffle split
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
#from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.cross_validation import StratifiedShuffleSplit
cross_validator = StratifiedShuffleSplit(y = labels_train, random_state = 0)

Pipeline = Pipeline([
        ('imp', Imputer()),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', GaussianNB())
    ])


param_grid = {'imp__strategy':['median', 'most_frequent'],
              #'clf__min_samples_split': [2, 3, 4, 5, 6, 10, 15, 20],
              #'clf__max_depth': [4, 5, 6, 7, 8],
              #'clf__n_estimators': [2,5,10,15],
              'pca__n_components': [2,3,4,5,6,7,8,9,10,11,12,13,14,15]}

#create the GridSearchCV object using param_grid
gridCV_object = GridSearchCV(estimator = Pipeline, 
                                        param_grid = param_grid,
                                        scoring = 'recall',
                                        cv = cross_validator)

#fit the model
gridCV_object.fit(features_train, labels_train)

#print the best model's parameters
print "Best parameters from the grid search:", pprint(gridCV_object.best_params_)

#instantiate the best model
clf_gridCV = gridCV_object.best_estimator_

#print out the best model's accuracy
print "\nBest estimator accuracy:", clf_gridCV.score(features_test, labels_test)

#get predictions and print out recall score
clf_gridCV_pred = clf_gridCV.predict(features_test)
print "\n\nRecall Score:", recall_score(labels_test, clf_gridCV_pred)

Best parameters from the grid search:{'imp__strategy': 'median', 'pca__n_components': 10}
 None

Best estimator accuracy: 0.818181818182


Recall Score: 0.2


Ok, Just going to do 12 components to search through. Also, I am reporting both the recall and the precision on this model. 

In [36]:
#Now using stratified shuffle split and the GaussianNB
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB

from sklearn.cross_validation import StratifiedShuffleSplit
cross_validator = StratifiedShuffleSplit(y = labels_train, random_state = 0)

Pipeline = Pipeline([
        ('imp', Imputer()),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', GaussianNB())
    ])


param_grid = {'imp__strategy':['median', 'most_frequent'],
              #'clf__min_samples_split': [2, 3, 4, 5, 6, 10, 15, 20],
              #'clf__max_depth': [4, 5, 6, 7, 8],
              #'clf__n_estimators': [2,5,10,15],
              'pca__n_components': [2,3,4,5,6,7,8,9,10,11,12]}

#create the GridSearchCV object using param_grid
gridCV_object = GridSearchCV(estimator = Pipeline, 
                                        param_grid = param_grid,
                                        scoring = 'recall',
                                        cv = cross_validator)

#fit the model
gridCV_object.fit(features_train, labels_train)

#print the best model's parameters
print "Best parameters from the grid search:", pprint(gridCV_object.best_params_)

#instantiate the best model
clf_gridCV = gridCV_object.best_estimator_

#print out the best model's accuracy
print "\nBest estimator accuracy:", clf_gridCV.score(features_test, labels_test)

#get predictions and print out recall score
clf_gridCV_pred = clf_gridCV.predict(features_test)
print "\n\nRecall Score:", recall_score(labels_test, clf_gridCV_pred)
print "\n\nPrecision Score:", precision_score(labels_test, clf_gridCV_pred)

Best parameters from the grid search:{'imp__strategy': 'median', 'pca__n_components': 10}
 None

Best estimator accuracy: 0.818181818182


Recall Score: 0.2


Precision Score: 0.2


Now we look at the preformance of this model. Now that we have fit the model we look at how well it does. First we print out the parameters of the model that was chosen by grid search as the best one. 

### Task 5: Tune precision and recall > 0.3

#### Standardize by subtracting mean and dividing by std_dev

In [11]:
from sklearn import preprocessing
# Create a minimum and maximum processor object
standard_scaler = preprocessing.StandardScaler()

# Create an object to transform the data to fit minmax processor
features_train = standard_scaler.fit_transform(features_train)
features_test = standard_scaler.fit_transform(features_test)

Now I am going to use the select k best method on the data and then see how it does with the models. 

COACH QUESTION: Where do I do the k-best thing? Can it be added to a pipeline below? 

In [12]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
test_classifier(clf, my_dataset, features_list, folds = 1000)

GaussianNB()
	Accuracy: 0.61633	Precision: 0.21514	Recall: 0.70900	F1: 0.33011	F2: 0.48592
	Total predictions: 15000	True positives: 1418	False positives: 5173	False negatives:  582	True negatives: 7827



COACH QUESTION: Why doesn't the rbf kernel work? 

In [13]:
from sklearn import svm
clf = svm.SVC(kernel='rbf',C=100) 
# tried gamma=0.001, C=100.
# kernel = 'linear', kernel='rbf'
test_classifier(clf, my_dataset, features_list, folds=1000)

Got a divide by zero when trying out: SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
Precision or recall may be undefined due to a lack of true positive predicitons.


In [14]:
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA(n_components=7)), ('svm', svm.SVC(kernel='rbf'))]
clf = Pipeline(estimators)
test_classifier(clf, my_dataset, features_list, folds = 1000)

Got a divide by zero when trying out: Pipeline(steps=[('reduce_dim', PCA(copy=True, n_components=7, whiten=False)), ('svm', SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
Precision or recall may be undefined due to a lack of true positive predicitons.


### The Winner
COACH QUESTION: So the Guassian Naive Bayes with the dimensions of the data set reduced to 8 through prinicple component analysis is the one that I got to break the 0.3 thresholds in Recall and Precision. I did it by just doing a manual search through the possible number of components. I have no idea which components were chosen and on what basis they worked, which 8 worked better than 9 or why 7 didn't make the 0.3 cut at all. I have no intuition for why any of this worked. And I don't know how to visualize it. Can I just submit this for the final project? 

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
estimators = [('reduce_dim', PCA(n_components=8)), ('Gaussian', GaussianNB())]
clf = Pipeline(estimators)
test_classifier(clf, my_dataset, features_list, folds = 1000)

Pipeline(steps=[('reduce_dim', PCA(copy=True, n_components=8, whiten=False)), ('Gaussian', GaussianNB())])
	Accuracy: 0.85053	Precision: 0.42568	Recall: 0.34650	F1: 0.38203	F2: 0.35989
	Total predictions: 15000	True positives:  693	False positives:  935	False negatives: 1307	True negatives: 12065

